In [1]:
import glob
import os
import tempfile
import sys

import anndata
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from ete3 import Tree

sys.path.insert(0, '/lab/solexa_weissman/kmin/git/KPTracer-release/cassiopeia-kp')
sys.path.append('/lab/solexa_weissman/kmin/kp_infercnv/NT')
from utilities import plot_tree_itol
from utilities import clonal_expansions

In [2]:
apiKey = ''
projectName = 'KP_Trees_Joseph' ## Specify the project name
plot_dir = 'plots'

In [3]:
tumor = '3513_NT_T3'

In [4]:
adata = anndata.read('/lab/solexa_weissman/mgjones/projects/kptc/RNA/NT/adata_processed.filtered.subcluster.nt.h5ad')
df_metadata = pd.read_csv('/lab/solexa_weissman/mgjones/projects/kptc/L6-37_ALL_META_052720.txt', sep='\t', index_col=0).dropna(subset=['Tumor', 'MetFamily'])

In [5]:
# Read chromosome sizes = 0 to end of last gene
df_genes = pd.read_csv('../mm10_gene_ordering_reordered.tsv', names=['gene', 'chr', 'start', 'end'], sep='\t', index_col=0)
chromosome_sizes = dict(df_genes.groupby('chr')['end'].max())
# Matplotlib can't plot very large images correctly, so we have to bin
# https://github.com/matplotlib/matplotlib/issues/19276
bin_size = 100
chromosome_order = [
    '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
    '16', '17', '18', '19', 'X', 'Y'
]
gene_positions = {chrom: {gene: ((row['start']-1) // bin_size, row['end'] // bin_size) for gene, row in df_chrom.iterrows()} for chrom, df_chrom in df_genes.groupby('chr')}

In [6]:
# Load Cassiopeia tree
df_model = pd.read_csv('/lab/solexa_weissman/mgjones/projects/kptc/trees/tumor_model.txt', sep='\t', index_col=0)
tree_cass = Tree(f'/lab/solexa_weissman/mgjones/projects/kptc/trees/{tumor}/{df_model.loc[tumor]["Newick"]}', format=1)
leaves = tree_cass.get_leaf_names()

genes = {}
chromosomes = {}
# for tumor in tumors:
print(tumor)
out_dir = f'{tumor}_grouped_0.2'

# Load CNVs
df_regions = pd.read_csv(f'{out_dir}/HMM_CNV_predictions.HMMi6.rand_trees.hmm_mode-subclusters.Pnorm_0.2.pred_cnv_regions.dat', sep='\t')
df_cells = pd.read_csv(f'{out_dir}/17_HMM_predHMMi6.rand_trees.hmm_mode-subclusters.cell_groupings', sep='\t')
df_merged = pd.merge(df_regions, df_cells[~df_cells['cell_group_name'].str.contains('Normal')], on='cell_group_name')

# Cells in tree that have CNVs
cells = set(df_merged['cell']).intersection(leaves)

tumor_chromosomes = {}
for cell, chrom, start, end, state in df_merged[df_merged['cell'].isin(cells)][['cell', 'chr', 'start', 'end', 'state']].values:
    start = (start-1) // bin_size
    end = end // bin_size
    chrom = str(chrom)

    tumor_chromosomes.setdefault(chrom, {}).setdefault(state, np.zeros((chromosome_sizes[chrom] - 1) // bin_size, dtype=np.uint16))
    tumor_chromosomes[chrom][state][start:end] += 1

    interval_genes = []
    for gene, (gene_start, gene_end) in gene_positions[chrom].items():
        if start <= gene_start and end >= gene_end:
            interval_genes.append(gene)
    genes.setdefault(tumor, {}).setdefault(cell, {}).update({gene: state for gene in interval_genes})
chromosomes[tumor] = tumor_chromosomes

dfs_counts = {}
for tumor, tumor_genes in genes.items():
    dfs_counts[tumor] = pd.DataFrame.from_dict(tumor_genes, orient='index').fillna(3).astype(int)

3513_NT_T3


In [7]:
def values_to_intervals(values):
    intervals = {}
    if len(values) == 0:
        return intervals
    
    start = 0
    prev_value = values[0]
    for i, value in enumerate(values):
        if value != prev_value:
            intervals[(start, i)] = prev_value
            start = i
            prev_value = value
    intervals[(start, len(values))] = prev_value
    return intervals

def collapse_intervals(intervals, wiggle=0.1):
    if not intervals:
        return intervals
    
    updated = True
    while updated:
        updated = False
        new_intervals = {}
        max_value = max(intervals.values())
        block_start = None
        block_end = None
        block_values = None
        for (start, end) in sorted(intervals.keys()):
            value = intervals[(start, end)]
            if block_start is None:
                block_start = start
                block_end = end
                block_values = [value]
                continue
            
            if block_end == start and abs(np.mean(block_values) - value) <= max_value * wiggle:
                block_end = end
                block_values.append(value)
                updated = True
            else:
                new_intervals[(block_start, block_end)] = np.mean(block_values)
                block_start = start
                block_end = end
                block_values = [value]
        new_intervals[(block_start, block_end)] = np.mean(block_values)
        intervals = new_intervals
    return new_intervals

# Collect all CNV counts
# This contains number of cells that have a CNV in each chromosome region
all_gains = {}
all_losses = {}
for tumor, tumor_chromosomes in chromosomes.items():
    for chrom, states in tumor_chromosomes.items():
        all_gains[chrom] = all_gains.get(
            chrom, np.zeros((chromosome_sizes[chrom] - 1) // bin_size, dtype=np.int32)
        ) + states.get(4, 0) + states.get(5, 0) + states.get(6, 0)
        all_losses[chrom] = all_losses.get(
            chrom, np.zeros((chromosome_sizes[chrom] - 1) // bin_size, dtype=np.int32)
        ) + states.get(1, 0) + states.get(2, 0)
        
# Select regions of interest from `all_gains` and `all_losses`
# A region is defined as interesting if at least 1% of cells have a CNV in that region
gain_regions = {}
loss_regions = {}
for chrom, gains in all_gains.items():
    for interval, count in collapse_intervals(values_to_intervals(gains)).items():
        if count > 0.1 * len(leaves):
            gain_regions.setdefault(chrom, {})[(interval)] = count
for chrom, losses in all_losses.items():
    for interval, count in collapse_intervals(values_to_intervals(losses)).items():
        if count > 0.1 * len(leaves):
            loss_regions.setdefault(chrom, {})[(interval)] = count

In [8]:
def state_color(state):
    cm = mpl.cm.seismic
    # Clip to [2, 4]
    state = min(max(state, 2), 4)
    return mpl.colors.to_hex(cm((state - 2) / 2))


temp_dir = tempfile.mkdtemp()
colorstrip_i = 0
colorstrips = {}
files = []    

# Expansions
path = os.path.join(temp_dir, 'expansions.txt')
header = f'DATASET_STYLE\nSEPARATOR TAB\nDATASET_LABEL\texpansions\nCOLOR\t#000000\nDATA\n\n'
expansion_exists = False
with open(path, 'w') as f:
    f.write(header)
    for _path in glob.glob(f'/lab/solexa_weissman/mgjones/projects/kptc/trees/{tumor}/clonal_expansions.{tumor}.expansion*.txt'):
        expansion_exists = True
        expanding_cells = list(pd.read_csv(_path, index_col=0, sep='\t')['0'])
        ancestor = tree_cass.get_common_ancestor(*expanding_cells)
        f.write(f'{ancestor.name}\tbranch\tclade\t#FF0000\t1\tnormal\n')
if expansion_exists:
    files.append(path)

# leiden coloring
path = os.path.join(temp_dir, '000_leiden.txt')
header = f'DATASET_COLORSTRIP\nSEPARATOR TAB\nCOLOR\t#000000\nDATASET_LABEL\tleiden\nSTRIP_WIDTH\t100\nMARGIN\t20\nSHOW_INTERNAL\t0\nDATA\n\n'
with open(path, 'w') as f:
    f.write(header)
    for cell in leaves:
        if cell in adata.obs.index:
            f.write(f'{cell}\t{adata.uns["leiden_sub_colors"][int(adata.obs.loc[cell]["leiden_sub"])]}\n')
        else:
            f.write(f'{cell}\t#FFFFFF\n')
files.append(path)

# CNVs
for chrom in chromosome_order:
    intervals = sorted(list(gain_regions.get(chrom, {})) + list(loss_regions.get(chrom, {})))
    
    for interval_start, interval_end in intervals:
        written = 0
        # Select genes within this interval
        interval_genes = []
        for gene, (gene_start, gene_end) in gene_positions[chrom].items():
            if interval_start <= gene_start and interval_end >= gene_end:
                interval_genes.append(gene)
                
        if not interval_genes:
            continue
        
        name = f'{str(colorstrip_i).zfill(2)}_{chrom}_{interval_start}_{interval_end}'
        path = os.path.join(temp_dir, f'{name}.txt')
        header = f'DATASET_COLORSTRIP\nSEPARATOR TAB\nCOLOR\t#000000\nDATASET_LABEL\t{name}\nSTRIP_WIDTH\t80\nMARGIN\t20\nSHOW_INTERNAL\t0\nDATA\n\n'
        with open(path, 'w') as f:
            f.write(header)

            df_counts = dfs_counts[tumor]
            df_counts = df_counts[df_counts.index.isin(leaves)]
            for cell, state in dict(df_counts[df_counts.columns[df_counts.columns.isin(interval_genes)]].mean(axis=1).dropna()).items():
                f.write(f'{cell}\t{state_color(state)}\n')
                written += 1
        if written > 0:
            files.append(path)
            colorstrips[colorstrip_i] = (chrom, interval_start, interval_end, interval_genes)
            colorstrip_i += 1
plot_tree_itol.upload_to_itol(
    tree_cass,
    apiKey,
    projectName,
    tumor,
    files=files,
    outfp=f'{plot_dir}/{tumor}_cnv_regions.pdf',
    rect=True,
    vertical_shift_factor=0.05,
    line_width=7,
)

iTOL output: SUCCESS: 1841168134311639431548

Tree Web Page URL: http://itol.embl.de/external.cgi?tree=1841168134311639431548&restore_saved=1
Warnings: []


In [19]:
for strip in colorstrips.values():
    print(f'Chr{strip[0]} {strip[1] * 100}-{strip[2] * 100}')

Chr6 70765700-71271600
Chr6 71271600-149101600
Chr12 3247400-75630500
Chr12 75630500-105040900
Chr12 105040900-105784600
Chr12 105784600-113153800
Chr15 3268500-101274700
Chr15 101284300-103244900
Chr17 6079700-34603600
Chr17 34603600-57065900
ChrX 7667900-134588000


In [16]:
from IPython.display import IFrame
IFrame(f'{plot_dir}/{tumor}_cnv_regions.pdf', width=500, height=600)